# Biostat M280 Homework 3

**Due Friday, May 25 @ 11:59PM**

## Q1 - Big $n$ regression

Those who took my _203B: Introduction to Data Science_ last quarter had a (painful) experience of wrangling an Apache Spark cluster to do linear regression on a dataset with more than 100 million observations. Now we learnt various methods for solving linear regression and should realize that, with right choice of algorithm, it is a problem that can be handled by any moderate computer.

### Q1(1)

Download the flight data from <http://stat-computing.org/dataexpo/2009/the-data.html>. For this exercise, we only need data from years 2003-2008. If you are using Mac or Linux, you can run the following Bash script, which downloads and unzips files for all years.
```bash
# Download flight data by year
for i in {1987..2008}
  do
    echo "$(date) $i Download"
    fnam=$i.csv.bz2
    wget -O ./$fnam http://stat-computing.org/dataexpo/2009/$fnam
    echo "$(date) $i unzip"
    bzip2 -d ./$fnam
  done

# Download airline carrier data
wget -O ./airlines.csv http://www.transtats.bts.gov/Download_Lookup.asp?Lookup=L_UNIQUE_CARRIERS

# Download airports data
wget -O ./airports.csv https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
```
Find out how many data points in each year.

### Q1(1) Solution:



In [1]:
# how many data points
@show countlines("2003.csv")
# how many data points
@show countlines("2004.csv")
# how many data points
@show countlines("2005.csv")
# how many data points
@show countlines("2006.csv")
# how many data points
@show countlines("2007.csv")
# how many data points
@show countlines("2008.csv");

countlines("2003.csv") = 6488541
countlines("2004.csv") = 7129271
countlines("2005.csv") = 7140597
countlines("2006.csv") = 7141923
countlines("2007.csv") = 7453216
countlines("2008.csv") = 7009729


### Q1(2) 

We are interested in how the time gain of a flight, defined as `DepDelay - ArrDelay`, depends on the distance traveled (`Distance`), departure delay (`DepDelay`), and carrier (`UniqueCarrier`). 

We want to fit a linear regression `Gain ~ 1 + Distance + DepDelay + UniqueCarrier` using data from 2003-2008. Note `UniqueCarrier` is a factor with 23 levels: "9E", "AA", "AQ", "AS", "B6", "CO", "DH", "DL", "EV", "F9", "FL", "HA", "HP", "MQ", "NW", "OH", "OO", "TZ", "UA", "US", "WN", "XE", and "YV". We use the dummy coding with "9E" as base level.

Will the design matrix (in double precision) fit into the memory of you computer?

### Q1(2) Solution:

First I checked the dimension of the design matrix we need for the regression. Since our model has an intercept, we have 5 columns and counting the number of rows in each year we have 41,490,375 rows. So our design matrix is a 41,490,375 by 5 matrix, and it can fit on the memory of my computer but I prefer it didn't... 

If each number is stored in double precicion, we need 8,629,998,000 bytes or 8.629998 GiB of memory to store this matrix.

In [1]:
41490375*26*8

8629998000

In [1]:
# import data from csv
using JuliaDB
# only need columns: DepDelay, ArrDelay, UniqueCarrier, Distance
@time table2003 = loadtable(
    "2003.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2003 = dropna(table2003)

@time table2004 = loadtable(
    "2004.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2004 = dropna(table2004)

@time table2005 = loadtable(
    "2005.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2005 = dropna(table2005)

@time table2006 = loadtable(
    "2006.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2006 = dropna(table2006)

@time table2007 = loadtable(
    "2007.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2007 = dropna(table2007)

@time table2008 = loadtable(
    "2008.csv", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
table2008 = dropna(table2008)

 48.374033 seconds (153.87 M allocations: 8.183 GiB, 26.93% gc time)
 20.021477 seconds (31.53 M allocations: 4.276 GiB, 7.28% gc time)
 19.828827 seconds (31.29 M allocations: 4.364 GiB, 7.48% gc time)
 21.101573 seconds (31.41 M allocations: 4.530 GiB, 18.85% gc time)
 23.794135 seconds (41.08 M allocations: 4.698 GiB, 21.54% gc time)
 36.608247 seconds (121.61 M allocations: 7.291 GiB, 31.04% gc time)


Table with 6855029 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
8         -14       "WN"           810
19        2         "WN"           810
8         14        "WN"           515
-4        -6        "WN"           515
34        34        "WN"           515
25        11        "WN"           688
67        57        "WN"           1591
-1        -18       "WN"           1591
2         2         "WN"           451
0         -16       "WN"           451
6         1         "WN"           828
94        80        "WN"           828
⋮
57        75        "DL"           481
80        99        "DL"           689
-2        15        "DL"           270
-4        6         "DL"           425
-3        16        "DL"           546
-1        2         "DL"           215
3         14        "DL"           533
-1        -2        "DL"           874
-5        0         "DL"           545
11        9         "DL"           533
7         -5  

In [1]:
# import data from csv
using JuliaDB
@time yrtable = loadtable(
    "allyears", 
    datacols = ["DepDelay", "ArrDelay", "UniqueCarrier", "Distance"])
# drop rows with missing values
yrtable = dropna(yrtable)

131.622394 seconds (388.14 M allocations: 17.700 GiB, 3.80% gc time)


Table with 41490375 rows, 4 columns:
DepDelay  ArrDelay  UniqueCarrier  Distance
───────────────────────────────────────────
-4        -1        "UA"           837
-1        -3        "UA"           837
29        23        "UA"           837
-2        -9        "UA"           1835
18        52        "UA"           1835
-4        6         "UA"           1835
-4        -8        "UA"           1835
0         2         "UA"           1835
-4        19        "UA"           1835
3         4         "UA"           413
-4        -23       "UA"           413
-3        -19       "UA"           413
⋮
57        75        "DL"           481
80        99        "DL"           689
-2        15        "DL"           270
-4        6         "DL"           425
-3        16        "DL"           546
-1        2         "DL"           215
3         14        "DL"           533
-1        -2        "DL"           874
-5        0         "DL"           545
11        9         "DL"           533
7        

After we drop missing values, I count again how many bytes this design matrix will use. Even after dropping rows with missing values, we will use 8.62999696 GiB of memory.

In [3]:
# how many data points
@show length(table2003)
# how many data points
@show length(table2004)
# how many data points
@show length(table2005)
# how many data points
@show length(table2006)
# how many data points
@show length(table2007)
# how many data points
@show length(table2008)

@show 41490375*26*8;

LoadError: [91mUndefVarError: table2003 not defined[39m

In [2]:
# mapping from variable names to X columns
# carrier "9E" is used as base level
const var2col = Dict(
        "Intercept" => 1,
        "Distance" => 2,
        "DepDelay" => 3,
        "AA" => 4,
        "AQ" => 5,
        "AS" => 6,
        "B6" => 7,
        "CO" => 8,
        "DH" => 9,
        "DL" => 10,
        "EV" => 11,
        "F9" => 12,
        "FL" => 13,
        "HA" => 14,
        "HP" => 15,
        "MQ" => 16,
        "NW" => 17,
        "OH" => 18,
        "OO" => 19,
        "TZ" => 20,
        "UA" => 21,
        "US" => 22,
        "WN" => 23,
        "XE" => 24,
        "YV" => 25,
        "Gain" => 26)
# mapping from column to variable names
const col2var = map(reverse, var2col)

# a custom function to generate [X y] from data table
function generate_xy(tbl::NextTable)
    # X matrix
    XY = zeros(length(tbl), 26)
    # intercept term
    @views fill!(XY[:, 1], 1)
    # Distance term
    @views copy!(XY[:, 2], columns(tbl, :Distance))
    # DepDelay term
    @views copy!(XY[:, 3], columns(tbl, :DepDelay))
    # Dummy coding for airline
    @inbounds for i in 1:length(tbl)
        yrtable[i][:UniqueCarrier] == "9E" && continue # base level
        XY[i, var2col[tbl[i][:UniqueCarrier]]] = 1
    end
    # last column is response: gain = depdelay - arrdelay
    XY[:, 26] = select(tbl, 
        (:DepDelay, :ArrDelay) => p -> Float64(p.DepDelay - p.ArrDelay))
    # return
    XY
end

function linreg_sweep_all(y::Vector, X::Matrix)
    n = size(X, 1)
    p = size(X, 2)
    
    tableau = [X y]' * [X y]
    sweep!(tableau, 1:p)
    beta_coeff_est = tableau[1:p, end]
    sigma_hat2 = tableau[end, end] / (n - 1) 
    SE_beta_coeff_est = sqrt.(Diagonal( .- sigma_hat2 .* tableau[1:p, 1:p]))
    return beta_coeff_est, sigma_hat2, SE_beta_coeff_est
end

linreg_sweep_all (generic function with 1 method)

In [3]:
xy = generate_xy(yrtable)

41490375×26 Array{Float64,2}:
 1.0   837.0  -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0   -3.0
 1.0   837.0  -1.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    2.0
 1.0   837.0  29.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    6.0
 1.0  1835.0  -2.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    7.0
 1.0  1835.0  18.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  -34.0
 1.0  1835.0  -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0  -10.0
 1.0  1835.0  -4.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0    4.0
 1.0  1835.0   0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   -2.0
 1.0  1835.0  -4.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0  -23.0
 1.0   413.0   3.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   -1.0
 1.0   413.0  -4.0  0.0  0.0  0.0  …  0.0  1.0  0.0  0.0  0.0  0.0   19.0
 1.0   413.0  -3.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   16.0
 1.0   413.0   0.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  0.0  0.0   12.0
 ⋮      

call sweep directly and you will get the desired quantities

In [4]:
using SweepOperator, BenchmarkTools

function linreg_sweep_all(y::Vector, X::Matrix)
    n = size(X, 1)
    p = size(X, 2)
    
    tableau = [X y]' * [X y]
    sweep!(tableau, 1:p)
    beta_coeff_est = tableau[1:p, end]
    sigma_hat2 = tableau[end, end] / (n - 1) 
    SE_beta_coeff_est = sqrt.(Diagonal( .- sigma_hat2 .* tableau[1:p, 1:p]))
    return beta_coeff_est, sigma_hat2, SE_beta_coeff_est
end

linreg_sweep_all (generic function with 1 method)

In [5]:
beta_coeff_est, sigma_hat2, SE_beta_coeff_est = linreg_sweep_all(xy[:, 26], xy[:, 1:25])

([1.14033, 0.00164935, -0.0118811, -1.8723, -0.5789, -0.938452, -1.42247, -2.57627, 1.16808, -2.19625  …  -1.46395, -3.62506, -0.00722279, -0.40365, -3.5774, -1.14816, -0.883753, 2.74855, -2.56721, -0.202211], 204.4031074256946, 
  [1 ,  1]  =  0.0202318
  [2 ,  2]  =  4.37958e-6
  [3 ,  3]  =  6.88878e-5
  [4 ,  4]  =  0.0215571
  [5 ,  5]  =  0.0521477
  [6 ,  6]  =  0.0250361
  [7 ,  7]  =  0.0259154
  [8 ,  8]  =  0.0229534
  [9 ,  9]  =  0.0266552
  [10, 10]  =  0.021659
  ⋮
  [15, 15]  =  0.0276918
  [16, 16]  =  0.0218055
  [17, 17]  =  0.0220088
  [18, 18]  =  0.023446
  [19, 19]  =  0.0217463
  [20, 20]  =  0.0375073
  [21, 21]  =  0.0219444
  [22, 22]  =  0.0220022
  [23, 23]  =  0.0209201
  [24, 24]  =  0.0222383
  [25, 25]  =  0.0255715)

### Q1(3)

Review the [Summary of Linear Regression](http://hua-zhou.github.io/teaching/biostatm280-2018spring/slides/12-linreg/linreg.html) and devise a strategy to solve the linear regression.

Report the estimated regression coefficients $\widehat \beta$, estimated variance $\widehat \sigma^2 = \sum_i (y_i - \widehat y_i)^2 / (n - 1)$, and coefficient standard errors.

Hint: It took my laptop less than 3 minutes to import data and fit linear regression.

### Q1(3) Solution:




In [27]:
beta_coeff_est

25-element Array{Float64,1}:
  1.14033   
  0.00164935
 -0.0118811 
 -1.8723    
 -0.5789    
 -0.938452  
 -1.42247   
 -2.57627   
  1.16808   
 -2.19625   
  1.03932   
 -2.15207   
 -1.35247   
 -1.87248   
 -0.350758  
 -1.46395   
 -3.62506   
 -0.00722279
 -0.40365   
 -3.5774    
 -1.14816   
 -0.883753  
  2.74855   
 -2.56721   
 -0.202211  

In [28]:
sigma_hat2

204.4031074256946

In [33]:
SE_beta_coeff_est

25×25 SparseMatrixCSC{Float64,Int64} with 25 stored entries:
  [1 ,  1]  =  0.0202318
  [2 ,  2]  =  4.37958e-6
  [3 ,  3]  =  6.88878e-5
  [4 ,  4]  =  0.0215571
  [5 ,  5]  =  0.0521477
  [6 ,  6]  =  0.0250361
  [7 ,  7]  =  0.0259154
  [8 ,  8]  =  0.0229534
  [9 ,  9]  =  0.0266552
  [10, 10]  =  0.021659
  ⋮
  [15, 15]  =  0.0276918
  [16, 16]  =  0.0218055
  [17, 17]  =  0.0220088
  [18, 18]  =  0.023446
  [19, 19]  =  0.0217463
  [20, 20]  =  0.0375073
  [21, 21]  =  0.0219444
  [22, 22]  =  0.0220022
  [23, 23]  =  0.0209201
  [24, 24]  =  0.0222383
  [25, 25]  =  0.0255715

### Q1(4)

Go to your resume/cv and claim you have experience performing analytics on data with hundred millions of observations.

### Q1(4) Solution:


I have edited my resume/cv and claimed my experience performing analysis on data with hundred millions of observations.

## Q2 - Google PageRank

We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

### Q2(1)

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write down the transition matrix $\mathbf{P}$ of the Markov chain as a diagonal matrix plus rank-1 matrix.

### Q2(1) Solution:

>Since with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page, if there are ${\sum_j a_{ij}}$ pages the probability of going to each one would be $\frac{1}{{\sum_j a_{ij}}}$.<br> 

>Additionally, we are given: if there are no links on the current page, with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page. If there are n total pages in the world, the probability of going to each one would be $1/n$.<br><br>

>We note that the transition probability of going from page i to page j is:
$$
\begin{eqnarray*}
    p_{ij} = \begin{cases}
    \frac{1}{\sum_j a_{ij}}*p + \frac{1}{n}*(1 - p) & \text{if page $i$ has links on the page, $r_i$ > 0} \\
    \frac{1}{n} & \text{if there are no links on the page $i$, $r_i$ = 0}
    \end{cases}
\end{eqnarray*}
$$<br>
> Replacing $r_i = \sum_j a_{ij}$ we have:
$$
\begin{eqnarray*}
    p_{ij} = \begin{cases}
    \frac{a_{ij}}{r_i}*p + \frac{1}{n}*(1 - p) & \text{if page $i$ has links on the page, $r_i$ > 0} \\
    \frac{1}{n} & \text{if there are no links on the page $i$, $r_i$ = 0}
    \end{cases}
\end{eqnarray*}
$$<br>

>We make note that the structure of transition matrix $P$ is the sum of two terms, a diagonal matrix + a rank-1 matrix, <br> where $\mathbf{\frac{1}{n}} = \frac{1}{n}\mathbf{1^T}$ and $\mathbf{\frac{1-p}{n}} = \frac{1-p}{n}\mathbf{1^T}$ are vectors. Then the resulting Transition Probability Matrix is:

\begin{eqnarray*}
    \mathbf{P} = \begin{cases}
     diag(\frac{p}{r_i})\mathbf{A} + \mathbf{\frac{1-p}{n}}\mathbf{1} & \text{if page $i$ has links on the page, $r_i$ > 0} \\
    diag(0)\mathbf{A} + \mathbf{\frac{1}{n}}\mathbf{1} & \text{if there are no links on the page $i$, $r_i$ = 0}
    \end{cases}
\end{eqnarray*}




### Q2(2)

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an eigen-problem. Show that it can also be cast as solving a linear system. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

### Q2(2) Solution:

>First we have:
$$\mathbf{P}^T \mathbf{x} = \mathbf{x}$$<br>
>Moving everything to the left side we have:
$$\mathbf{P}^T \mathbf{x} - \mathbf{x} = \mathbf{0}$$<br>
>We can re-write this as:
$$(\mathbf{I} - \mathbf{P}^T) \mathbf{x} = \mathbf{0}$$<br>
>Now, we see that this problem can also be cast as solving a linear system. What we want to find is the solution `x`.
>If we let $\mathbf{C} = (\mathbf{I} - \mathbf{P}^T),$ we know $\mathbf{C}$ is not full rank. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient and we know each column of $\mathbf{C}$ will also sum to 1.<br><br>
>Thus, we can replace the first equation by the $\sum_{i=1}^n x_i = 1$ by replacing the first row of $\mathbf{C}$ with a vector of ones, and create a new vector for the solution, where the first postition is a 1 (indicating we changed the 1st row of matrix $\mathbf{C}$ to ones) and the rest are zeros. <br><br>
>Note that we could choose any of the n rows of matrix $\mathbf{C}$ to be ones, adjusting the corresponding solution vector with a 1 in the position of the row number that was changed.

### Q2(3)

Download the [`ucla.zip`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw3/ucla.zip) package from course webpage. Unzip the package, which contains two files `U.txt` and `A.txt`. `U.txt` lists the 500 URL names. `A.txt` is the $500 \times 500$ connectivity matrix. Read data into Julia. Compute summary statistics:
* number of pages
* number of edges
* number of dangling nodes (pages with no out links)
* which page has max in-degree?
* which page has max out-degree?
* visualize the sparsity pattern of $\mathbf{A}$

### Q2(3) Solution:

In [37]:
# connectivity matrix
A = readcsv("A.txt")
# Make diagonals 0 so that probability of going from page i to page i is 0 !
A = A - Diagonal(A)
# Let Julia know A is a sparse matrix since most entries are 0's!
A = sparse(A);
# number of pages
@show n_pages = size(A, 1)
# number of edges (page links)
@show n_edges = countnz(A)
# number of dangling nodes (pages with no out links)
r = sum(A, 2) #number of outlinks per page
@show n_dangling_nodes = n_pages - countnz(r) # total number of pages minus those with out links

n_pages = size(A, 1) = 500
n_edges = countnz(A) = 10853
n_dangling_nodes = n_pages - countnz(r) = 103


103

In [17]:
# U = readcsv("U.txt")
U = readlines("U.txt")

500-element Array{String,1}:
 "http://www.ucla.edu"                                                                                                                     
 "http://4393665.fls.doubleclick.net/activityi;src=4393665;type=Prima0;cat=UCLAe0;ord="                                                    
 "http://www.atmos.ucla.edu/weather/quick-look"                                                                                            
 "http://bit.ly/10Wf0Te"                                                                                                                   
 "http://bit.ly/1NMd3KF"                                                                                                                   
 "http://bit.ly/1JTGEHi"                                                                                                                   
 "http://bit.ly/1JTGEHj"                                                                                                           

There are 500 pages in the U.txt file. 

### Q2(4)

Set the _teleportation_ parameter at $p = 0.85$. Try the following methods to solve for $\mathbf{x}$ using the `ucla.zip` data.

0. A dense linear system solver such as LU decomposition.  
0. A simple iterative linear system solver such as Jacobi or Gauss-Seidel.   
0. A dense eigen-solver.  
0. A simple iterative eigen-solver such as the power method.  

For iterative methods, you can use the [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package. Make sure to utilize the special structure of $\mathbf{P}$ (diagonal + rank 1) to speed up the matrix-vector multiplication.

## Q2(4) Solution:

### Using LU-decomposition

In [18]:
#first make P
# Matrix P is the sum of a sparse matrix + a rank 1 matrix
n = size(A, 1)
p = 0.85
r = sum(A, 2) #sum each row of A to get r_i for each page i
P = zeros(n, n)
term1 = zeros(n)
term2 = repeat([1/n], outer=[n]) 
for i in 1:n
    if r[i] > 0 
        term1[i] = p / r[i]
        term2[i] = (1 - p) / n
    end
end
#first make P^T
P_transpose = (Diagonal(term1) * sparse(A) .+ term2*(ones(1, n))).'

#then make C = (I- P^t)
C = I - P_transpose
C[1, :] = ones(n)      
b = zeros(n)
b[1] = 1.0

x_old = ones(n, 1) / n
x_new = zeros(n)
function LU_Decomposition_Linear_Solver(C, b)
#use the fact that each column sums to 1 and replace the first row of C with ones
#then solve x = C \ [1, 0, ... , 0] 
x = C \ b
    return x
end

LU_Decomposition_Linear_Solver (generic function with 1 method)

In [19]:
x_LU = LU_Decomposition_Linear_Solver(C, b)

500-element Array{Float64,1}:
 0.0127701 
 0.00130393
 0.00405405
 0.00398338
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00145083
 0.00141309
 ⋮         
 0.0011959 
 0.00149245
 0.0011959 
 0.00345944
 0.00211007
 0.0011959 
 0.0011959 
 0.0084935 
 0.0011959 
 0.00149103
 0.0011959 
 0.0011959 

### Q2(5)

List the top 20 ranked URLs you found.

first concatenate 1:n and then sort and then get indices

In [155]:
using DataFrames
indices = collect(1:n)
x_with_indices = hcat(x, indices)

500×2 Array{Float64,2}:
 0.0127701     1.0
 0.00130393    2.0
 0.00405405    3.0
 0.00398338    4.0
 0.00130393    5.0
 0.00130393    6.0
 0.00130393    7.0
 0.00130393    8.0
 0.00130393    9.0
 0.00130393   10.0
 0.00130393   11.0
 0.00145083   12.0
 0.00141309   13.0
 ⋮                
 0.0011959   489.0
 0.00149245  490.0
 0.0011959   491.0
 0.00345944  492.0
 0.00211007  493.0
 0.0011959   494.0
 0.0011959   495.0
 0.0084935   496.0
 0.0011959   497.0
 0.00149103  498.0
 0.0011959   499.0
 0.0011959   500.0

In [135]:
sorted = sortrows(x_with_indices, rev = true)

500×2 Array{Float64,2}:
 0.0228785    459.0
 0.0193529     64.0
 0.0127701      1.0
 0.0115253     27.0
 0.0107311     29.0
 0.0084935    496.0
 0.00810174   462.0
 0.00796066    36.0
 0.00754457    28.0
 0.00676869    43.0
 0.00653717    57.0
 0.00653717    60.0
 0.00652746    62.0
 ⋮                 
 0.00075957    80.0
 0.000756004  348.0
 0.000756004  345.0
 0.000756004  340.0
 0.000756004  342.0
 0.000756004  352.0
 0.000756004  336.0
 0.000756004  350.0
 0.000754551  405.0
 0.000748165  408.0
 0.000746597  134.0
 0.000746597   85.0

We note that there is a 4 way tie between the 19th - 22nd sites.

In [145]:
top22 = sorted[1:22, 1:2]

22×2 Array{Float64,2}:
 0.0228785   459.0
 0.0193529    64.0
 0.0127701     1.0
 0.0115253    27.0
 0.0107311    29.0
 0.0084935   496.0
 0.00810174  462.0
 0.00796066   36.0
 0.00754457   28.0
 0.00676869   43.0
 0.00653717   57.0
 0.00653717   60.0
 0.00652746   62.0
 0.00652746   63.0
 0.00652746   61.0
 0.0065169    55.0
 0.0065169    54.0
 0.0065169    59.0
 0.00650538   56.0
 0.00650538   58.0
 0.00650538   53.0
 0.00650538   51.0

In [161]:
top22_indices = convert(Array{Int64}, top22[1:22, 2])
top22_pages = U[top22_indices]

22-element Array{Any,1}:
 "http://schema.org"                                                                     
 "http://www"                                                                            
 "http://www.ucla.edu"                                                                   
 "http://www.directory.ucla.edu"                                                         
 "http://www.universityofcalifornia.edu"                                                 
 "http://schema.org/CreativeWork"                                                        
 "http://schema.org/Text"                                                                
 "http://giveto.ucla.edu"                                                                
 "http://www.registrar.ucla.edu/calendar"                                                
 "http://www.uclalumni.net"                                                              
 "http://www.magazine.ucla.edu/depts/lifesigns/honoring-adolescence"       

## Jacobi Iterative Method

>Note that for our problem we are finding the solution $\mathbf{x}$ to the equation $\mathbf{C}\mathbf{x} = \mathbf{b},$
where $\mathbf{C} = (\mathbf{I} - \mathbf{P^{T}})$

$$x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} c_{ij} x_j^{(t)} - \sum_{j=i+1}^n c_{ij} x_j^{(t)}}{c_{ii}}$$

Splitting up Matrix A into the sum of a Lower Triangular, Diagonal and Upper triangular matrix we have: 
$$\mathbf{C} = \mathbf{L} + \mathbf{D} + \mathbf{U}$$<br>
$$x_i^{(t+1)} = -\mathbf{D^{-1}}\mathbf{C}x_i^{(t)} + x_i^{(t)} + \mathbf{D^{-1}}\mathbf{b}$$

>Now substituting back in $\mathbf{C} = (\mathbf{I} - \mathbf{P^{T}})$ we have:
$$x_i^{(t+1)} = -\mathbf{D^{-1}}(\mathbf{I} - \mathbf{P^{T}})x_i^{(t)} + x_i^{(t)} + \mathbf{D^{-1}}\mathbf{b}$$




In [24]:
function Jacobi_Linear_Solver(A::AbstractMatrix, telep::Float64 = 0.85; 
        maxiter::Int = 10000, tolerance::Float64 = 1e-9, 
        x_old::Vector = repeat([1/size(A, 1)], outer=[size(A, 1)]))
#first make P
# Matrix P is the sum of a sparse matrix + a rank 1 matrix
n = size(A, 1)
r = sum(A, 2) #sum each row of A to get r_i for each page i
P = zeros(n, n)
term1 = zeros(n)
term2 = repeat([1/n], outer=[n]) 
for i in 1:n
    if r[i] > 0 
        term1[i] = telep / r[i]
        term2[i] = (1 - telep) / n
    end
end
#first make P^T
P_transpose = (Diagonal(term1) * sparse(A) .+ term2*(ones(1, n))).'

#then make C = (I- P^t)
C = I - P_transpose
b = zeros(n)
    
x_new = zeros(n)
x_old = ones(n, 1) / n

D_inv = inv(Diagonal(C)) 
for i in 1:maxiter
x_new = -D_inv * (C * x_old) .+ x_old .+ (D_inv * b)
    
if vecnorm(x_new - x_old) < tolerance
        break
         end
     end
 return x_new
end

Jacobi_Linear_Solver (generic function with 2 methods)

In [25]:
x_jacobi = Jacobi_Linear_Solver(A)

500×1 Array{Float64,2}:
 0.015434   
 0.000831992
 0.0045823  
 0.00451133 
 0.000831992
 0.000831992
 0.000831992
 0.000831992
 0.000831992
 0.000831992
 0.000831992
 0.00100879 
 0.00109823 
 ⋮          
 0.000920927
 0.00143108 
 0.000920927
 0.00381179 
 0.00150396 
 0.000920927
 0.000920927
 0.0111382  
 0.000920927
 0.00142913 
 0.000920927
 0.000919089

In [26]:
vecnorm(x_LU - x_jacobi)

0.015755321026345477

# Dense Eigen-Solver

It is well-known that x is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix  P.  That is $P^{T}x = x$. Therefore x can be solved as an eigen-problem.
 
We first perform eigen decomposition on $P^{T}$ to get the left eigenvector corresponding to the top eigenvalue 1, and find vector $x_i$, the proportion of times the surfer visits page $i$ in the long run. Therefore `x_eigen_solvers` serves as page ranks.

In [192]:
# eig-decomposition C
C = eigfact((P_transpose)) # this gets the eigen decomp of P^T which has the eigen values and vectors

# we get the real elements of the first column of eigenvectors 
eigenvals_weights = real(C[:vectors][:, 1]) 

# use eigen solvers to get the long run frequency of each page
x_eigen_solvers = eigenvals_weights / sum(eigenvals_weights) # find the proportion of time spent in each page over total time

500-element Array{Float64,1}:
 0.0127701 
 0.00130393
 0.00405405
 0.00398338
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00130393
 0.00145083
 0.00141309
 ⋮         
 0.0011959 
 0.00149245
 0.0011959 
 0.00345944
 0.00211007
 0.0011959 
 0.0011959 
 0.0084935 
 0.0011959 
 0.00149103
 0.0011959 
 0.0011959 

### Q2(6)

As of Monday May 11 2018, there are at least 1.83 billion indexed webpages on internet according to <http://www.worldwidewebsize.com/>. Explain whether each of these methods works for the PageRank problem at this scale.